In [ ]:
# pip install PyGithub

## Process multiple files sequentially & Extract images and graphics URLs

In [1]:
import requests
import json
import csv
import os
import re
import time  # For adding delays between requests

API_TOKEN = 'add figma token'
FILE_KEYS = ['XN57QSSgnI4exbB2OTK1QM/MODULE1', '5cH2oMyNQPZ1w9fUPM9Mfe/MODULE2']  # Ensure MODULE is part of the file key or name

# Figma API URL template
FIGMA_API_URL_TEMPLATE = 'https://api.figma.com/v1/files/{}'

# Headers for the API request
headers = {
    'X-Figma-Token': API_TOKEN
}

# Base URL for the GitHub repository to fetch images and graphics
GITHUB_BASE_URL = 'https://raw.githubusercontent.com/UNDP-Data/dsc-energy-academy-pipeline/main/03_Inputs/'

# Function to determine the module (section) number from the file name or URL (e.g., MODULE1 -> 1)
def get_section_number(file_name):
    match = re.search(r'MODULE(\d+)', file_name, re.IGNORECASE)
    return int(match.group(1)) if match else None

# Function to determine the appropriate header tag based on font size
def determine_header_tag(font_size):
    if font_size >= 32:
        return 'h1'
    elif 24 <= font_size < 32:
        return 'h2'
    elif 20 <= font_size < 24:
        return 'h3'
    elif 16 <= font_size < 20:
        return 'h4'
    elif 14 <= font_size < 16:
        return 'h5'
    else:
        return 'h6'

# Function to determine the template type based on frame content
def determine_template_type(frame):
    contains_text = False
    contains_image = False
    label_like_text_only = True  # Assume the text is a label by default

    for node in frame['children']:
        if node['type'] == 'TEXT':
            text = node.get('characters', '').strip()
            # Check if the text is more than a label (like image_1 or graph_1)
            if re.search(r'image_\d+|graph_\d+', text):
                continue  # Ignore label-like text
            elif len(text) > 0:  # Consider non-empty and non-label text as real text
                contains_text = True  # Regular text, not a label
                label_like_text_only = False
        if node['type'] in ['IMAGE', 'VECTOR', 'RECTANGLE']:  # 'VECTOR' or 'RECTANGLE' might be used for images
            contains_image = True

    # Now determine template type based on whether the frame contains real text or just label-like text
    if contains_image and label_like_text_only:  # Only label-like text and image
        return 'Photo'
    elif contains_image and contains_text:
        return 'Text-Image'
    elif contains_text and not contains_image:
        return 'Text'
    else:
        return 'Unknown'  # In case neither text nor image is found


# Function to process a frame and extract relevant information
def process_frame(frame, section_number, subsection_number=None):
    # Check if 'absoluteBoundingBox' exists
    if 'absoluteBoundingBox' not in frame:
        print(f"Warning: 'absoluteBoundingBox' missing for frame '{frame.get('name', 'Unnamed')}'. Skipping this frame.")
        return None  # Return None to indicate skipping this frame
    
    # Determine template type (Photo/Text or Text-Image) based on content
    template_type = determine_template_type(frame)
    
    frame_data = {
        'Section': section_number,
        'Subsection': subsection_number, 
        'Section Title': frame['name'],  
        'Template Type': template_type,
        'Column': 0,  
        'Text': '',  
        'x': frame['absoluteBoundingBox']['x'],  # Capture the x position for sorting
        'y': frame['absoluteBoundingBox']['y'],  # Capture the y position for sorting
        'image_url': None  # Placeholder for the image URL
    }

    # Process all nodes within the frame
    for node in frame['children']:
        if node['type'] == 'TEXT':
            font_size = node.get('style', {}).get('fontSize', 14)  # Default to 14 if no fontSize is found
            header_tag = determine_header_tag(font_size)
            text = node['characters']
            # Concatenate text with the appropriate header tags
            frame_data['Text'] += f'<{header_tag}>{text}</{header_tag}>'

            # Add the image/graphic URL if image_[number] or graph_[number] is found
            image_url = get_image_url(text, section_number)
            if image_url:
                frame_data['image_url'] = image_url
    
    return frame_data


# Function to recursively extract frames from the file data and assign subsections
def extract_frames(nodes, section_number=1):
    frames = []  # Initialize a new list for each call to avoid data mix-up
    
    # Extract all frames in a flat list
    for node in nodes:
        if node['type'] == 'FRAME':
            # Process the frame with the current section and default subsection number
            processed_frame = process_frame(node, section_number)
            if processed_frame:  # Only add if the frame was successfully processed
                frames.append(processed_frame)
        
        # Recursively process child nodes (if any)
        if 'children' in node:
            frames.extend(extract_frames(node['children'], section_number))

    return frames

# Function to dynamically assign subsections based on similar y-values (rows)
def assign_subsections_by_row(frames, y_threshold=100):
    if not frames:
        return frames

    # Sort frames by their y-position first, then by x-position
    frames.sort(key=lambda f: (f['y'], f['x']))

    subsection_number = 1
    last_y = frames[0]['y']
    
    for frame in frames:
        # If the vertical difference between frames is larger than the threshold, start a new subsection
        if abs(frame['y'] - last_y) > y_threshold:
            subsection_number += 1

        # Assign the subsection number
        frame['Subsection'] = subsection_number

        # Update last_y to the current frame's y-position
        last_y = frame['y']
    
    return frames

# Function to fetch image and graphic URL from GitHub
def get_image_url(text, section_number, extension='.png'):
    # Check if the text contains image or graph labels, and fetch the corresponding URL
    image_match = re.search(r'image_(\d+)', text)
    graph_match = re.search(r'graph_(\d+)', text)
    
    if image_match:
        return f'{GITHUB_BASE_URL}Images/image_{image_match.group(1)}{extension}'
    elif graph_match:
        return f'{GITHUB_BASE_URL}Graphics/Module{section_number}/graph_{graph_match.group(1)}{extension}'
    
    return None


# Function to save the frames data to a CSV file
def save_to_csv(frames, filename):
    # Assign the correct column number after sorting
    for i, frame in enumerate(frames):
        frame['Column'] = i + 1
        frame.pop('x', None)  # Remove 'x' after sorting
        frame.pop('y', None)  # Remove 'y' after sorting

    # Specify the CSV column headers
    csv_columns = ['Section', 'Subsection', 'Section Title', 'Template Type', 'Column', 'Text', 'image_url']
    
    # Define the path to save the CSV file in the "01_Results" folder
    results_folder = os.path.join('..', '01_Results')
    os.makedirs(results_folder, exist_ok=True)  # Ensure the folder exists

    # Full path to the CSV file
    csv_path = os.path.join(results_folder, filename)
    
    # Open the CSV file for writing
    with open(csv_path, 'w', newline='', encoding='utf-8') as csv_file:
        writer = csv.DictWriter(csv_file, fieldnames=csv_columns)
        writer.writeheader()
        
        # Write each frame's data to the CSV file
        for frame in frames:
            writer.writerow(frame)

# Function to save the frames data to JSON files
def save_to_json(frames, filename):
    # Assign the correct column number after sorting
    for i, frame in enumerate(frames):
        frame['Column'] = i + 1
        frame.pop('x', None)  # Remove 'x' after sorting
        frame.pop('y', None)  # Remove 'y' after sorting
    
    # Define the path to save the JSON file in the "00_API" folder
    api_folder = os.path.join('..', '00_API')
    os.makedirs(api_folder, exist_ok=True)  # Ensure the folder exists


    # Full path to the JSON file
    json_path = os.path.join(api_folder, filename)                   

    # Convert the frames list to a JSON structure and save to file
    with open(json_path, 'w', encoding='utf-8') as json_file:
        json.dump(frames, json_file, indent=4, ensure_ascii=False)
        
# Adding delay and retry logic for handling rate limits and avoiding timeouts
def fetch_file_data(file_key):
    max_retries = 5
    retry_count = 0
    delay_between_retries = 10  # delay of 10 seconds
    while retry_count < max_retries:
        response = requests.get(FIGMA_API_URL_TEMPLATE.format(file_key.split('/')[0]), headers=headers)
        if response.status_code == 200:
            return response.json()
        elif response.status_code == 429:  # Rate limit hit, Figma API returns 429
            print("Rate limit hit. Waiting for rate limit reset...")
            retry_after = int(response.headers.get('Retry-After', delay_between_retries))
            time.sleep(retry_after)
            retry_count += 1
        else:
            print(f"Error: Failed to fetch data for file key {file_key}. Status code: {response.status_code}")
            retry_count += 1
            time.sleep(delay_between_retries)
    print(f"Max retries reached for file {file_key}. Skipping...")
    return None

# Main processing function with sequential fetching and retry handling
def process_figma_files(file_keys):
    for file_key in file_keys:
        section_number = get_section_number(file_key) or 0
        print(f"Processing file for Module {section_number}:")

        file_data = fetch_file_data(file_key)
        if file_data and 'document' in file_data:
            frames = extract_frames(file_data['document']['children'], section_number)
            
            # Assign subsections based on rows (y-coordinate differences)
            frames = assign_subsections_by_row(frames, y_threshold=100)

            csv_filename = f'figma_frames_{file_key.split("/")[0]}.csv'
            json_filename = f'figma_frames_{file_key.split("/")[0]}.json'

            save_to_csv(frames, csv_filename)
            print(f"Frames data has been exported to {csv_filename}")

            save_to_json(frames, json_filename)
            print(f"Frames data has been exported to {json_filename}")
        else:
            print(f"Warning: 'document' key not found in file {file_key}. Skipping.")

# Execute the main function with the list of file keys
process_figma_files(FILE_KEYS)


Processing file for Module 1:
Frames data has been exported to figma_frames_XN57QSSgnI4exbB2OTK1QM.csv
Frames data has been exported to figma_frames_XN57QSSgnI4exbB2OTK1QM.json
Processing file for Module 2:
Frames data has been exported to figma_frames_5cH2oMyNQPZ1w9fUPM9Mfe.csv
Frames data has been exported to figma_frames_5cH2oMyNQPZ1w9fUPM9Mfe.json
